In [2]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Initialize the Gemini model
gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

In [3]:
# Sample HTML template
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{title}}</title>
    <style>
        body {
            font-family: {{font_family}};
            background-color: {{background_color}};
            color: {{text_color}};
            margin: 0;
            padding: 20px;
        }
        header {
            text-align: center;
            padding: 20px;
            background-color: {{header_color}};
            color: white;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }
        .main-content {
            padding: 20px;
            background-color: white;
            border-radius: 5px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
        }
        footer {
            text-align: center;
            margin-top: 20px;
            padding: 10px;
            background-color: {{footer_color}};
            color: white;
        }
    </style>
</head>
<body>
    <header>
        <h1>{{header_text}}</h1>
    </header>
    <div class="container">
        <div class="main-content">
            <h2>{{main_heading}}</h2>
            <p>{{main_content}}</p>
        </div>
    </div>
    <footer>
        <p>{{footer_text}}</p>
    </footer>
</body>
</html>
"""

In [4]:

# Define questions to ask the user
def get_user_preferences():
    questions = [
        "What is the title for your webpage?",
        "What main heading would you like to display?",
        "Please provide the main content for your page:",
        "What text would you like in the header?",
        "What text would you like in the footer?",
        "Choose a primary color scheme (e.g., blue, green, red, purple):",
        "Would you prefer a light or dark theme?",
        "What font style would you prefer (e.g., modern, classic, playful)?",
    ]
    
    answers = {}
    print("Please answer the following questions to customize your HTML template:")
    for question in questions:
        answers[question] = input(f"{question} ")
    
    return answers

# Use Gemini to process user preferences and generate style recommendations
def generate_style_recommendations(user_preferences):
    # Define response schemas for structured output
    response_schemas = [
        ResponseSchema(name="font_family", description="Font family recommendation based on user's style preference"),
        ResponseSchema(name="background_color", description="Background color in hex code"),
        ResponseSchema(name="text_color", description="Text color in hex code"),
        ResponseSchema(name="header_color", description="Header background color in hex code"),
        ResponseSchema(name="footer_color", description="Footer background color in hex code"),
    ]
    
    # Create output parser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()
    
    # Create prompt template
    template = """
    Based on the following user preferences, recommend appropriate styles for a webpage:
    
    Color scheme preference: {color_scheme}
    Theme preference: {theme}
    Font style preference: {font_style}
    
    {format_instructions}
    """
    
    prompt = PromptTemplate(
        template=template,
        input_variables=["color_scheme", "theme", "font_style"],
        partial_variables={"format_instructions": format_instructions}
    )
    
    # Format the prompt with user preferences
    formatted_prompt = prompt.format(
        color_scheme=user_preferences["Choose a primary color scheme (e.g., blue, green, red, purple):"],
        theme=user_preferences["Would you prefer a light or dark theme?"],
        font_style=user_preferences["What font style would you prefer (e.g., modern, classic, playful)?"]
    )
    
    # Get response from Gemini
    response = gemini.invoke(formatted_prompt)
    
    # Parse the response
    try:
        return output_parser.parse(response.content)
    except Exception as e:
        print(f"Error parsing Gemini response: {e}")
        # Fallback values
        return {
            "font_family": "'Arial', sans-serif",
            "background_color": "#f4f4f4",
            "text_color": "#333333",
            "header_color": "#4a6fa5",
            "footer_color": "#4a6fa5"
        }

# Replace placeholders in the template with user values
def customize_template(template, user_preferences, style_recommendations):
    customized_html = template
    
    # Map user answers to template placeholders
    replacements = {
        "{{title}}": user_preferences["What is the title for your webpage?"],
        "{{main_heading}}": user_preferences["What main heading would you like to display?"],
        "{{main_content}}": user_preferences["Please provide the main content for your page:"],
        "{{header_text}}": user_preferences["What text would you like in the header?"],
        "{{footer_text}}": user_preferences["What text would you like in the footer?"],
        "{{font_family}}": style_recommendations["font_family"],
        "{{background_color}}": style_recommendations["background_color"],
        "{{text_color}}": style_recommendations["text_color"],
        "{{header_color}}": style_recommendations["header_color"],
        "{{footer_color}}": style_recommendations["footer_color"]
    }
    
    # Perform replacements
    for placeholder, value in replacements.items():
        customized_html = customized_html.replace(placeholder, value)
    
    return customized_html

# Save the customized HTML to a file
def save_html_file(html_content, filename="customized_template.html"):
    with open(filename, "w") as file:
        file.write(html_content)
    print(f"\nHTML saved to {filename}")
    return filename



In [5]:
# Main function
def main():
    print("Welcome to the HTML Template Customizer!\n")
    
    # Get user preferences
    user_preferences = get_user_preferences()
    
    # Generate style recommendations based on preferences
    print("\nGenerating style recommendations...")
    style_recommendations = generate_style_recommendations(user_preferences)
    
    # Customize the template
    print("Customizing your template...")
    customized_html = customize_template(html_template, user_preferences, style_recommendations)
    
    # Save to file
    filename = save_html_file(customized_html)
    
    # Print summary
    print("\nTemplate customization complete!")
    print(f"Your template has been customized with the following settings:")
    print(f"Title: {user_preferences['What is the title for your webpage?']}")
    print(f"Theme: {user_preferences['Would you prefer a light or dark theme?']}")
    print(f"Color Scheme: {user_preferences['Choose a primary color scheme (e.g., blue, green, red, purple):']}")
    print(f"Font Style: {style_recommendations['font_family']}")
    
    # Return the HTML
    return customized_html

if __name__ == "__main__":
    # Execute the main function
    customized_html = main()
    print("\nHere is your customized HTML:\n")
    print(customized_html)

Welcome to the HTML Template Customizer!

Please answer the following questions to customize your HTML template:

Generating style recommendations...
Customizing your template...

HTML saved to customized_template.html

Template customization complete!
Your template has been customized with the following settings:
Title: Zamlik
Theme: dark
Color Scheme: blue
Font Style: Roboto, sans-serif

Here is your customized HTML:


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Zamlik</title>
    <style>
        body {
            font-family: Roboto, sans-serif;
            background-color: #121212;
            color: #FFFFFF;
            margin: 0;
            padding: 20px;
        }
        header {
            text-align: center;
            padding: 20px;
            background-color: #1E3A8A;
            color: white;
        }
        .container {
            max-width: 1200px;
     